{
    "Does the privacy policy affirm that the company implements appropriate technical security measures?": {
        "policy_data": {
			// this bit is the policy hash
            "fac08b738949bae239a1ea2e33ba9508b1abc152fb00bd2bd2e836986c3bc67c": { 
				// this hash is currently irrelecant
                "da75684459834dd6459a7c2a3d407ec73ae99680e2d7540a2c6127e76d17374c": [
                    {
                        "substring": "We implement appropriate technical and organizational security measures designed to protect personal data from loss, misuse, and unauthorized access, disclosure, alteration, or destruction.",
                        "substring_indices": [
                            20045,
                            20234
                        ],
                        "method": "FACT_EMBEDDINGS"
                    }
                ]
            }
        }, /* other bits of data */
    
    }, /* other questions */

}


{
    "openai": {
        "64b194a739dade62a6c90a81d803b42e298c2dcccb79a1f1fcca83df888ad67c": {
            "policy_content": "# Europe privacy policy\n\nFor ....",
			/* other data */,
			"fetch_date": "2025-11-23 06:42:43.738333"
		},
		/* other versions */
	},
	/* other policies */
}


In [1]:
import os
import json

QUESTIONS_FILE = "./data/questions_filter_after.json"
POLICIES_FILE = "./data/policies_testing.json"
OUTPUT_Q_FILE = "./output_q.json"
OUTPUT_P_FILE = "./output_p.json"


def _loadJson(filepath):
	if not os.path.exists(filepath):
		print(f"Warning: File not found: {filepath}")
		return {}
	try:
		with open(filepath, "r", encoding="utf-8") as f:
			return json.load(f)
	except json.JSONDecodeError:
		print(f"Error decoding JSON: {filepath}")
		return {}


qdata = _loadJson(QUESTIONS_FILE)
pdata = _loadJson(POLICIES_FILE)

In [20]:
import numpy as np
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis

data_set = qdata
strings = list(data_set.keys())

# raw_vectors = np.array(
# 	[data_set[s]["retrieval_embedding_vector"] for s in strings]
# )	# alt
raw_vectors = np.array([data_set[s]["embedding_vector"] for s in strings])


print(f"Original shape: {raw_vectors.shape}")


truncated_vectors = raw_vectors[:, :256]


norms = np.linalg.norm(truncated_vectors, axis=1, keepdims=True)

cleaned_vectors = truncated_vectors / (norms + 1e-10)

print(f"Truncated shape: {cleaned_vectors.shape}")


lw = LedoitWolf()
lw.fit(cleaned_vectors)


precision_matrix = lw.precision_
mean_vector = lw.location_


results = []

# for i, string_key in enumerate(strings):
# 	vec = cleaned_vectors[i]

# 	dist = mahalanobis(vec, mean_vector, precision_matrix)

# 	results.append((dist, string_key))


# results.sort(key=lambda x: x[0])

# print("\n--- The Two Most Representative Strings (Closest to Centroid) ---")
# for i in range(2):
# 	dist, s = results[i]
# 	print(f'Rank {i+1} (Distance: {dist:.4f}):\n   "{s}"\n')


# print("\n--- The Least Representative String (Outlier) ---")
# dist, s = results[-1]
# print(f'Last Rank (Distance: {dist:.4f}):\n   "{s}"\n')


def get_pairwise_distance(key1, key2, string_list, vector_array, inv_cov_matrix):

	try:

		idx1 = string_list.index(key1)
		idx2 = string_list.index(key2)

		vec1 = vector_array[idx1]
		vec2 = vector_array[idx2]

		dist = mahalanobis(vec1, vec2, inv_cov_matrix)

		print(f"Comparison Results:")
		print(f"Statement A: {key1}")
		print(f"Statement B: {key2}")
		print(f"Mahalanobis Distance: {dist:.4f}")

		euclidean = np.linalg.norm(vec1 - vec2)
		print(f"Standard Euclidean:   {euclidean:.4f}")

		return dist

	except ValueError:
		print("Error: One of the keys was not found in the string list.")
		return None


key_a = "Does the privacy policy affirm that security measures are designed to protect personal data from misuse?"
key_b = "Does the privacy policy affirm that security measures are designed to protect personal data from unauthorized access?"


distance = get_pairwise_distance(
	key_a, key_b, strings, cleaned_vectors, precision_matrix
)
import itertools

print("\n--- Searching for the absolute closest pairs (True Duplicates) ---")

min_dist = float("inf")
closest_pair = None
all_distances = []

# Efficiently calculate all pairs (assuming N=520, N^2 is tiny for modern CPUs)
# We only check the upper triangle to avoid double counting
num_vectors = len(cleaned_vectors)
for i in range(num_vectors):
	for j in range(i + 1, num_vectors):
		vec_i = cleaned_vectors[i]
		vec_j = cleaned_vectors[j]

		# Calculate Mahalanobis
		delta = vec_i - vec_j
		d = mahalanobis(vec_i, vec_j, precision_matrix)

		all_distances.append(d)

		if d < min_dist:
			min_dist = d
			closest_pair = (strings[i], strings[j])

print(f"Minimum Distance found in entire dataset: {min_dist:.4f}")
print(f"String A: {closest_pair[0]}")
print(f"String B: {closest_pair[1]}")

# ---------------------------------------------------------
# 3. Calculate the Percentile of your "Misuse vs Access" pair
# ---------------------------------------------------------
# This tells you: Is 14.4 "close" or "far" relative to the rest of the data?

target_dist = 14.5802
percentile = sum(1 for x in all_distances if x < target_dist) / len(all_distances) * 100

print(f"\n--- Context for 'Misuse vs Access' {target_dist} ---")
print(f"This pair is closer than {100 - percentile:.2f}% of all other pairs.")
if percentile < 5:
	print("Verdict: Extremely Similar (Likely Duplicate)")
elif percentile < 20:
	print("Verdict: Somewhat Similar (Related Concepts)")
else:
	print("Verdict: Distinct (Different Concepts)")

import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist, mahalanobis
from sklearn.cluster import AgglomerativeClustering

DISTANCE_THRESHOLD = 9.22

print(f"Computing pairwise Mahalanobis distances for {len(cleaned_vectors)} vectors...")
print(f"Using Threshold: {DISTANCE_THRESHOLD}")

dist_matrix = cdist(
	cleaned_vectors, cleaned_vectors, metric="mahalanobis", VI=precision_matrix
)
cluster_model = AgglomerativeClustering(
	n_clusters=None,
	distance_threshold=DISTANCE_THRESHOLD,
	metric="precomputed",
	linkage="complete",
)

labels = cluster_model.fit_predict(dist_matrix)


df = pd.DataFrame(
	{"string": strings, "label": labels, "vector_index": range(len(strings))}
)

all_clusters = [group for _, group in df.groupby("label")]

significant_clusters = [g for g in all_clusters if len(g) > 1]

significant_clusters.sort(key=lambda x: len(x), reverse=True)

total_groups = len(all_clusters)
trivial_count = total_groups - len(significant_clusters)

print(f"\nResult: Found {len(significant_clusters)} groups containing duplicates.")
print(f"(Hidden {trivial_count} unique items that had no matches within threshold)\n")
print("=" * 80)


# for i, group_df in enumerate(significant_clusters):

# 	indices = group_df["vector_index"].tolist()
# 	cluster_strings = group_df["string"].tolist()

# 	print(f"GROUP {i+1} (Size: {len(indices)})")

# 	cluster_vectors = cleaned_vectors[indices]

# 	local_mean = np.mean(cluster_vectors, axis=0)

# 	min_dist = float("inf")
# 	rep_index = -1

# 	for idx, vec_idx in enumerate(indices):
# 		d = mahalanobis(cleaned_vectors[vec_idx], local_mean, precision_matrix)
# 		if d < min_dist:
# 			min_dist = d
# 			rep_index = idx

# 	# Print strings, highlighting the representative
# 	for idx, s in enumerate(cluster_strings):
# 		if idx == rep_index:
# 			print(f" [CENTROID]  {s}")
# 		else:
# 			print(f"             {s}")

# 	print("-" * 80)

for i, group_df in enumerate(significant_clusters):

	indices = group_df["vector_index"].tolist()
	cluster_strings = group_df["string"].tolist()

	# 1. Calculate Centroid
	cluster_vectors = cleaned_vectors[indices]
	local_mean = np.mean(cluster_vectors, axis=0)

	# 2. Calculate distances of all members to the centroid
	distances_to_centroid = []
	min_dist = float("inf")
	rep_index = -1

	for idx, vec_idx in enumerate(indices):
		d = mahalanobis(cleaned_vectors[vec_idx], local_mean, precision_matrix)
		distances_to_centroid.append(d)

		# Track the centroid representative
		if d < min_dist:
			min_dist = d
			rep_index = idx

	# 3. Radius is the maximum distance from the center
	# (How far does this cluster stretch?)
	group_radius = max(distances_to_centroid)

	# Print Header with Radius
	print(f"GROUP {i+1} (Size: {len(indices)}) [Radius: {group_radius:.4f}]")

	# Print strings, highlighting the representative
	for idx, s in enumerate(cluster_strings):
		if idx == rep_index:
			print(f" [CENTROID]  {s}")
		else:
			print(f"             {s}")

	print("-" * 80)

Original shape: (520, 3072)
Truncated shape: (520, 256)
Comparison Results:
Statement A: Does the privacy policy affirm that security measures are designed to protect personal data from misuse?
Statement B: Does the privacy policy affirm that security measures are designed to protect personal data from unauthorized access?
Mahalanobis Distance: 14.4228
Standard Euclidean:   0.3790

--- Searching for the absolute closest pairs (True Duplicates) ---
Minimum Distance found in entire dataset: 8.3352
String A: Does the privacy policy affirm that the company collects the dates and times of access?
String B: Does the privacy policy affirm that the company collects the dates and times of access to the services?

--- Context for 'Misuse vs Access' 14.5802 ---
This pair is closer than 99.83% of all other pairs.
Verdict: Extremely Similar (Likely Duplicate)
Computing pairwise Mahalanobis distances for 520 vectors...
Using Threshold: 9.22

Result: Found 5 groups containing duplicates.
(Hidden 510 

In [22]:
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering

DISTANCE_THRESHOLD = 9.223086354550212
# DISTANCE_THRESHOLD = 8.823086354550213	# 9.223086354550212

print(f"Computing pairwise Mahalanobis distances for {len(cleaned_vectors)} vectors...")
print(f"Using Threshold: {DISTANCE_THRESHOLD}")

dist_matrix = cdist(
	cleaned_vectors, cleaned_vectors, metric="mahalanobis", VI=precision_matrix
)
cluster_model = AgglomerativeClustering(
	n_clusters=None,
	distance_threshold=DISTANCE_THRESHOLD,
	metric="precomputed",
	linkage="complete",
)

labels = cluster_model.fit_predict(dist_matrix)


df = pd.DataFrame(
	{"string": strings, "label": labels, "vector_index": range(len(strings))}
)

all_clusters = [group for _, group in df.groupby("label")]

significant_clusters = [g for g in all_clusters if len(g) > 1]

significant_clusters.sort(key=lambda x: len(x), reverse=True)

total_groups = len(all_clusters)
trivial_count = total_groups - len(significant_clusters)

baseline_indices = {
	tuple(g["vector_index"].sort_values().tolist()) for g in significant_clusters
}

if not baseline_indices:
	print(
		"Baseline run resulted in no significant clusters (all size 1 or 0). Skipping stability check."
	)
else:
	print(
		f"\nStarting stability check against {len(baseline_indices)} baseline significant clusters."
	)

	STEP_SIZE = 0.05	# Define a reasonable step size
	MAX_ITERATIONS = 500

	t = DISTANCE_THRESHOLD
	final_stable_threshold = DISTANCE_THRESHOLD

	for i in range(MAX_ITERATIONS):
		t += STEP_SIZE

		cluster_model_t = AgglomerativeClustering(
			n_clusters=None,
			distance_threshold=t,
			metric="precomputed",
			linkage="complete",
		)

		labels_t = cluster_model_t.fit_predict(dist_matrix)

		df_t = pd.DataFrame(
			{"string": strings, "label": labels_t, "vector_index": range(len(strings))}
		)

		all_clusters_t = [group for _, group in df_t.groupby("label")]
		significant_clusters_t = [g for g in all_clusters_t if len(g) > 1]

		current_indices = {
			tuple(g["vector_index"].sort_values().tolist()) for g in significant_clusters_t
		}

		if not baseline_indices.issubset(current_indices):

			print("\n--- STABILITY BROKEN ---")
			print(f"First merge/change detected at proposed threshold t={t:.6f}")
			print(f"Maximum stable threshold found: {final_stable_threshold:.6f}")
			break

		else:
			final_stable_threshold = t
			if i % 20 == 0:
				print(
					f"Iteration {i}: Threshold {t:.6f} stable. Significant clusters found: {len(significant_clusters_t)}"
				)

	else:
		print(
			f"\nReached Max Iterations ({MAX_ITERATIONS}). Final stable threshold: {final_stable_threshold:.6f}"
		)

Computing pairwise Mahalanobis distances for 520 vectors...
Using Threshold: 9.223086354550212

Starting stability check against 5 baseline significant clusters.
Iteration 0: Threshold 9.273086 stable. Significant clusters found: 5
Iteration 20: Threshold 10.273086 stable. Significant clusters found: 11
Iteration 40: Threshold 11.273086 stable. Significant clusters found: 19
Iteration 60: Threshold 12.273086 stable. Significant clusters found: 37
Iteration 80: Threshold 13.273086 stable. Significant clusters found: 54
Iteration 100: Threshold 14.273086 stable. Significant clusters found: 77
Iteration 120: Threshold 15.273086 stable. Significant clusters found: 105

--- STABILITY BROKEN ---
First merge/change detected at proposed threshold t=15.323086
Maximum stable threshold found: 15.273086


In [6]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist, mahalanobis
from sklearn.cluster import AgglomerativeClustering

DISTANCE_THRESHOLD = 9.22

print(f"Computing pairwise Mahalanobis distances for {len(cleaned_vectors)} vectors...")
print(f"Using Threshold: {DISTANCE_THRESHOLD}")

dist_matrix = cdist(
	cleaned_vectors, cleaned_vectors, metric="mahalanobis", VI=precision_matrix
)
cluster_model = AgglomerativeClustering(
	n_clusters=None,
	distance_threshold=DISTANCE_THRESHOLD,
	metric="precomputed",
	linkage="complete",
)

labels = cluster_model.fit_predict(dist_matrix)


df = pd.DataFrame(
	{"string": strings, "label": labels, "vector_index": range(len(strings))}
)

all_clusters = [group for _, group in df.groupby("label")]

significant_clusters = [g for g in all_clusters if len(g) > 1]

significant_clusters.sort(key=lambda x: len(x), reverse=True)

total_groups = len(all_clusters)
trivial_count = total_groups - len(significant_clusters)

print(f"\nResult: Found {len(significant_clusters)} groups containing duplicates.")
print(f"(Hidden {trivial_count} unique items that had no matches within threshold)\n")
print("=" * 80)

Computing pairwise Mahalanobis distances for 520 vectors...
Using Threshold: 9.22

Result: Found 7 groups containing duplicates.
(Hidden 506 unique items that had no matches within threshold)



In [ ]:
def get_pairwise_distance(key1, key2, string_list, vector_array, inv_cov_matrix):

	try:

		idx1 = string_list.index(key1)
		idx2 = string_list.index(key2)

		vec1 = vector_array[idx1]
		vec2 = vector_array[idx2]

		dist = mahalanobis(vec1, vec2, inv_cov_matrix)

		print(f"Comparison Results:")
		print(f"Statement A: {key1}")
		print(f"Statement B: {key2}")
		print(f"Mahalanobis Distance: {dist:.4f}")

		euclidean = np.linalg.norm(vec1 - vec2)
		print(f"Standard Euclidean:   {euclidean:.4f}")

		return dist

	except ValueError:
		print("Error: One of the keys was not found in the string list.")
		return None


key_a = "Does the privacy policy affirm that security measures are designed to protect personal data from misuse?"
key_b = "Does the privacy policy affirm that security measures are designed to protect personal data from unauthorized access?"


distance = get_pairwise_distance(
	key_a, key_b, strings, cleaned_vectors, precision_matrix
)
import itertools

print("\n--- Searching for the absolute closest pairs (True Duplicates) ---")

min_dist = float("inf")
closest_pair = None
all_distances = []

# Efficiently calculate all pairs (assuming N=520, N^2 is tiny for modern CPUs)
# We only check the upper triangle to avoid double counting
num_vectors = len(cleaned_vectors)
for i in range(num_vectors):
	for j in range(i + 1, num_vectors):
		vec_i = cleaned_vectors[i]
		vec_j = cleaned_vectors[j]

		# Calculate Mahalanobis
		delta = vec_i - vec_j
		d = mahalanobis(vec_i, vec_j, precision_matrix)

		all_distances.append(d)

		if d < min_dist:
			min_dist = d
			closest_pair = (strings[i], strings[j])

print(f"Minimum Distance found in entire dataset: {min_dist:.4f}")
print(f"String A: {closest_pair[0]}")
print(f"String B: {closest_pair[1]}")

# ---------------------------------------------------------
# 3. Calculate the Percentile of your "Misuse vs Access" pair
# ---------------------------------------------------------
# This tells you: Is 14.4 "close" or "far" relative to the rest of the data?

target_dist = 14.5802
percentile = sum(1 for x in all_distances if x < target_dist) / len(all_distances) * 100

print(f"\n--- Context for 'Misuse vs Access' {target_dist} ---")
print(f"This pair is closer than {100 - percentile:.2f}% of all other pairs.")
if percentile < 5:
	print("Verdict: Extremely Similar (Likely Duplicate)")
elif percentile < 20:
	print("Verdict: Somewhat Similar (Related Concepts)")
else:
	print("Verdict: Distinct (Different Concepts)")

import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist, mahalanobis
from sklearn.cluster import AgglomerativeClustering

DISTANCE_THRESHOLD = 11.0

print(f"Computing pairwise Mahalanobis distances for {len(cleaned_vectors)} vectors...")
print(f"Using Threshold: {DISTANCE_THRESHOLD}")

dist_matrix = cdist(
	cleaned_vectors, cleaned_vectors, metric="mahalanobis", VI=precision_matrix
)
cluster_model = AgglomerativeClustering(
	n_clusters=None,
	distance_threshold=DISTANCE_THRESHOLD,
	metric="precomputed",
	linkage="complete",
)

labels = cluster_model.fit_predict(dist_matrix)


df = pd.DataFrame(
	{"string": strings, "label": labels, "vector_index": range(len(strings))}
)

all_clusters = [group for _, group in df.groupby("label")]

significant_clusters = [g for g in all_clusters if len(g) > 1]

significant_clusters.sort(key=lambda x: len(x), reverse=True)

total_groups = len(all_clusters)
trivial_count = total_groups - len(significant_clusters)

print(f"\nResult: Found {len(significant_clusters)} groups containing duplicates.")
print(f"(Hidden {trivial_count} unique items that had no matches within threshold)\n")
print("=" * 80)


for i, group_df in enumerate(significant_clusters):

	indices = group_df["vector_index"].tolist()
	cluster_strings = group_df["string"].tolist()

	print(f"GROUP {i+1} (Size: {len(indices)})")

	cluster_vectors = cleaned_vectors[indices]

	local_mean = np.mean(cluster_vectors, axis=0)

	min_dist = float("inf")
	rep_index = -1

	for idx, vec_idx in enumerate(indices):
		d = mahalanobis(cleaned_vectors[vec_idx], local_mean, precision_matrix)
		if d < min_dist:
			min_dist = d
			rep_index = idx

	# Print strings, highlighting the representative
	for idx, s in enumerate(cluster_strings):
		if idx == rep_index:
			print(f" [CENTROID]  {s}")
		else:
			print(f"             {s}")

	print("-" * 80)

Comparison Results:
Statement A: Does the privacy policy affirm that security measures are designed to protect personal data from misuse?
Statement B: Does the privacy policy affirm that security measures are designed to protect personal data from unauthorized access?
Mahalanobis Distance: 14.4228
Standard Euclidean:   0.3790


In [29]:
import itertools

print("\n--- Searching for the absolute closest pairs (True Duplicates) ---")

min_dist = float("inf")
closest_pair = None
all_distances = []

# Efficiently calculate all pairs (assuming N=520, N^2 is tiny for modern CPUs)
# We only check the upper triangle to avoid double counting
num_vectors = len(cleaned_vectors)
for i in range(num_vectors):
	for j in range(i + 1, num_vectors):
		vec_i = cleaned_vectors[i]
		vec_j = cleaned_vectors[j]

		# Calculate Mahalanobis
		delta = vec_i - vec_j
		d = mahalanobis(vec_i, vec_j, precision_matrix)

		all_distances.append(d)

		if d < min_dist:
			min_dist = d
			closest_pair = (strings[i], strings[j])

print(f"Minimum Distance found in entire dataset: {min_dist:.4f}")
print(f"String A: {closest_pair[0]}")
print(f"String B: {closest_pair[1]}")

# ---------------------------------------------------------
# 3. Calculate the Percentile of your "Misuse vs Access" pair
# ---------------------------------------------------------
# This tells you: Is 14.4 "close" or "far" relative to the rest of the data?

target_dist = 14.5802
percentile = sum(1 for x in all_distances if x < target_dist) / len(all_distances) * 100

print(f"\n--- Context for 'Misuse vs Access' {target_dist} ---")
print(f"This pair is closer than {100 - percentile:.2f}% of all other pairs.")
if percentile < 5:
	print("Verdict: Extremely Similar (Likely Duplicate)")
elif percentile < 20:
	print("Verdict: Somewhat Similar (Related Concepts)")
else:
	print("Verdict: Distinct (Different Concepts)")


--- Searching for the absolute closest pairs (True Duplicates) ---
Minimum Distance found in entire dataset: 8.3352
String A: Does the privacy policy affirm that the company collects the dates and times of access?
String B: Does the privacy policy affirm that the company collects the dates and times of access to the services?

--- Context for 'Misuse vs Access' 14.5802 ---
This pair is closer than 99.83% of all other pairs.
Verdict: Extremely Similar (Likely Duplicate)


In [ ]:
import numpy as np
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis

data_set = qdata
strings = list(data_set.keys())

raw_vectors = np.array(
	[data_set[s]["retrieval_embedding_vector"] for s in strings]
)	# alt
# raw_vectors = np.array([data_set[s]["embedding_vector"] for s in strings])


print(f"Original shape: {raw_vectors.shape}")


truncated_vectors = raw_vectors[:, :256]


norms = np.linalg.norm(truncated_vectors, axis=1, keepdims=True)

cleaned_vectors = truncated_vectors / (norms + 1e-10)

print(f"Truncated shape: {cleaned_vectors.shape}")


lw = LedoitWolf()
lw.fit(cleaned_vectors)


precision_matrix = lw.precision_
mean_vector = lw.location_


results = []


DISTANCE_THRESHOLD = 14.873086	# 8.823086354550213

print(f"Computing pairwise Mahalanobis distances for {len(cleaned_vectors)} vectors...")
print(f"Using Threshold: {DISTANCE_THRESHOLD}")

dist_matrix = cdist(
	cleaned_vectors, cleaned_vectors, metric="mahalanobis", VI=precision_matrix
)
cluster_model = AgglomerativeClustering(
	n_clusters=None,
	distance_threshold=DISTANCE_THRESHOLD,
	metric="precomputed",
	linkage="complete",
)

labels = cluster_model.fit_predict(dist_matrix)


df = pd.DataFrame(
	{"string": strings, "label": labels, "vector_index": range(len(strings))}
)

all_clusters = [group for _, group in df.groupby("label")]

significant_clusters = [g for g in all_clusters if len(g) > 1]

significant_clusters.sort(key=lambda x: len(x), reverse=True)

total_groups = len(all_clusters)
trivial_count = total_groups - len(significant_clusters)

Original shape: (520, 3072)
Truncated shape: (520, 256)
Computing pairwise Mahalanobis distances for 520 vectors...
Using Threshold: 14.873086


In [12]:
print(f"\nResult: Found {len(significant_clusters)} groups containing duplicates.")
print(f"(Hidden {trivial_count} unique items that had no matches within threshold)\n")
print("=" * 80)


for i, group_df in enumerate(significant_clusters):

	indices = group_df["vector_index"].tolist()
	cluster_strings = group_df["string"].tolist()

	print(f"GROUP {i+1} (Size: {len(indices)})")

	cluster_vectors = cleaned_vectors[indices]

	local_mean = np.mean(cluster_vectors, axis=0)

	min_dist = float("inf")
	rep_index = -1

	for idx, vec_idx in enumerate(indices):
		d = mahalanobis(cleaned_vectors[vec_idx], local_mean, precision_matrix)
		if d < min_dist:
			min_dist = d
			rep_index = idx

	# Print strings, highlighting the representative
	for idx, s in enumerate(cluster_strings):
		if idx == rep_index:
			print(f" [CENTROID]  {s}")
		else:
			print(f"             {s}")

	print("-" * 80)


Result: Found 99 groups containing duplicates.
(Hidden 299 unique items that had no matches within threshold)

GROUP 1 (Size: 5)
             Does the privacy policy affirm that user names are entrusted to the domestic representative?
             Does the privacy policy affirm that user IDs are entrusted to the domestic representative?
             Does the privacy policy affirm that user phone numbers are entrusted to the domestic representative?
 [CENTROID]  Does the privacy policy affirm that user email addresses are entrusted to the domestic representative?
             Does the privacy policy affirm that user addresses are entrusted to the domestic representative?
--------------------------------------------------------------------------------
GROUP 2 (Size: 4)
             Does the privacy policy affirm that users have the statutory right to withdraw consent at any time where consent is the legal basis for processing?
 [CENTROID]  Does the privacy policy affirm that users have 

In [9]:
significant_clusters

[                                                string  label  vector_index
 158  Does the privacy policy affirm that providing ...      0           158
 159  Does the privacy policy affirm that providing ...      0           159,
                                                string  label  vector_index
 52  Does the privacy policy affirm that the compan...      1            52
 85  Does the privacy policy affirm that the compan...      1            85,
                                                 string  label  vector_index
 337  Does the privacy policy affirm that users have...      2           337
 363  Does the privacy policy affirm that users have...      2           363,
                                                 string  label  vector_index
 302  Does the privacy policy affirm that the compan...      3           302
 442  Does the privacy policy affirm that the compan...      3           442,
                                                 string  label  vector_inde

In [19]:
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering


DISTANCE_THRESHOLD = 8.823086354550213	# 9.223086354550212

print(f"Computing pairwise Mahalanobis distances for {len(cleaned_vectors)} vectors...")
print(f"Using Threshold: {DISTANCE_THRESHOLD}")

dist_matrix = cdist(
	cleaned_vectors, cleaned_vectors, metric="mahalanobis", VI=precision_matrix
)
cluster_model = AgglomerativeClustering(
	n_clusters=None,
	distance_threshold=DISTANCE_THRESHOLD,
	metric="precomputed",
	linkage="complete",
)

labels = cluster_model.fit_predict(dist_matrix)


df = pd.DataFrame(
	{"string": strings, "label": labels, "vector_index": range(len(strings))}
)

all_clusters = [group for _, group in df.groupby("label")]

significant_clusters = [g for g in all_clusters if len(g) > 1]

significant_clusters.sort(key=lambda x: len(x), reverse=True)

total_groups = len(all_clusters)
trivial_count = total_groups - len(significant_clusters)

baseline_indices = {
	tuple(g["vector_index"].sort_values().tolist()) for g in significant_clusters
}

if not baseline_indices:
	print(
		"Baseline run resulted in no significant clusters (all size 1 or 0). Skipping stability check."
	)
else:
	print(
		f"\nStarting stability check against {len(baseline_indices)} baseline significant clusters."
	)

	STEP_SIZE = 0.05	# Define a reasonable step size
	MAX_ITERATIONS = 500

	t = DISTANCE_THRESHOLD
	final_stable_threshold = DISTANCE_THRESHOLD

	for i in range(MAX_ITERATIONS):
		t += STEP_SIZE

		cluster_model_t = AgglomerativeClustering(
			n_clusters=None,
			distance_threshold=t,
			metric="precomputed",
			linkage="complete",
		)

		labels_t = cluster_model_t.fit_predict(dist_matrix)

		df_t = pd.DataFrame(
			{"string": strings, "label": labels_t, "vector_index": range(len(strings))}
		)

		all_clusters_t = [group for _, group in df_t.groupby("label")]
		significant_clusters_t = [g for g in all_clusters_t if len(g) > 1]

		current_indices = {
			tuple(g["vector_index"].sort_values().tolist()) for g in significant_clusters_t
		}

		if not baseline_indices.issubset(current_indices):

			print("\n--- STABILITY BROKEN ---")
			print(f"First merge/change detected at proposed threshold t={t:.6f}")
			print(f"Maximum stable threshold found: {final_stable_threshold:.6f}")
			break

		else:
			final_stable_threshold = t
			if i % 20 == 0:
				print(
					f"Iteration {i}: Threshold {t:.6f} stable. Significant clusters found: {len(significant_clusters_t)}"
				)

	else:
		print(
			f"\nReached Max Iterations ({MAX_ITERATIONS}). Final stable threshold: {final_stable_threshold:.6f}"
		)

Computing pairwise Mahalanobis distances for 520 vectors...
Using Threshold: 8.823086354550213

Starting stability check against 5 baseline significant clusters.
Iteration 0: Threshold 8.873086 stable. Significant clusters found: 5
Iteration 20: Threshold 9.873086 stable. Significant clusters found: 15
Iteration 40: Threshold 10.873086 stable. Significant clusters found: 24
Iteration 60: Threshold 11.873086 stable. Significant clusters found: 37
Iteration 80: Threshold 12.873086 stable. Significant clusters found: 54
Iteration 100: Threshold 13.873086 stable. Significant clusters found: 79
Iteration 120: Threshold 14.873086 stable. Significant clusters found: 99

--- STABILITY BROKEN ---
First merge/change detected at proposed threshold t=15.573086
Maximum stable threshold found: 15.523086


In [17]:
5 / ((99 + 105) / 2)

0.049019607843137254

In [13]:
import itertools

print("\n--- Searching for the absolute closest pairs (True Duplicates) ---")

min_dist = float("inf")
closest_pair = None
all_distances = []

# Efficiently calculate all pairs (assuming N=520, N^2 is tiny for modern CPUs)
# We only check the upper triangle to avoid double counting
num_vectors = len(cleaned_vectors)
for i in range(num_vectors):
	for j in range(i + 1, num_vectors):
		vec_i = cleaned_vectors[i]
		vec_j = cleaned_vectors[j]

		# Calculate Mahalanobis
		delta = vec_i - vec_j
		d = mahalanobis(vec_i, vec_j, precision_matrix)

		all_distances.append(d)

		if d < min_dist:
			min_dist = d
			closest_pair = (strings[i], strings[j])

print(f"Minimum Distance found in entire dataset: {min_dist:.4f}")
print(f"String A: {closest_pair[0]}")
print(f"String B: {closest_pair[1]}")

# ---------------------------------------------------------
# 3. Calculate the Percentile of your "Misuse vs Access" pair
# ---------------------------------------------------------
# This tells you: Is 14.4 "close" or "far" relative to the rest of the data?

target_dist = 14.4228
percentile = sum(1 for x in all_distances if x < target_dist) / len(all_distances) * 100

print(f"\n--- Context for 'Misuse vs Access' (14.42) ---")
print(f"This pair is closer than {100 - percentile:.2f}% of all other pairs.")
if percentile < 5:
	print("Verdict: Extremely Similar (Likely Duplicate)")
elif percentile < 20:
	print("Verdict: Somewhat Similar (Related Concepts)")
else:
	print("Verdict: Distinct (Different Concepts)")


--- Searching for the absolute closest pairs (True Duplicates) ---
Minimum Distance found in entire dataset: 5.7731
String A: Does the privacy policy affirm that users have the right to request the correction of inaccurate personal data?
String B: Does the privacy policy affirm that users have the right to request the correction of inaccurate data?

--- Context for 'Misuse vs Access' (14.42) ---
This pair is closer than 99.83% of all other pairs.
Verdict: Extremely Similar (Likely Duplicate)


In [ ]:
def _saveJson(filepath, data):
	with open(filepath, "w", encoding="utf-8") as f:
		json.dump(data, f, indent=4, default=str)
	print(f"Saved {filepath}")


def preprocessStatement(statement):
	"""
	Converts "Does the privacy policy affirm..." to "The privacy policy affirms..."
	"""
	prefix = "Does the privacy policy affirm"
	if statement.startswith(prefix):
		processed_text = ("The privacy policy affirms" + statement[len(prefix) :])[:-1] + "."
		return processed_text
	return statement


qdata = _loadJson(QUESTIONS_FILE)
pdata = _loadJson(POLICIES_FILE)
q_output = {}

for question, data in qdata.items():
	statement = preprocessStatement(question)

	if statement not in q_output:
		q_output[statement] = {}

	if "policy_data" in data:
		for p_hash, inner_data in data["policy_data"].items():

			if p_hash not in q_output[statement]:
				q_output[statement][p_hash] = []

			for _, occurrences in inner_data.items():
				for occurrence in occurrences:
					q_output[statement][p_hash].append(
						{"substring_indices": occurrence["substring_indices"]}
					)

p_output = {}

for policy_name, versions in pdata.items():
	p_output[policy_name] = {}

	for policy_hash, details in versions.items():
		# Map the fields we need
		p_output[policy_name][policy_hash] = {
			"policy_content": details.get("policy_content", ""),
			"fetch_date": details.get("fetch_date", ""),
		}

print(len(list(qdata.keys())))
# _saveJson(OUTPUT_Q_FILE, q_output)
# _saveJson(OUTPUT_P_FILE, p_output)

520


In [ ]:
# import os
# import json


# def _loadJson(filepath):
# 	if not os.path.exists(filepath):
# 		return {}
# 	try:
# 		with open(filepath, "r", encoding="utf-8") as f:
# 			return json.load(f)
# 	except json.JSONDecodeError:
# 		return {}


# QUESTIONS_FILE = "./questions_filter_after.json"
# POLICIES_FILE = "./policies_testing.json"

# qdata = _loadJson(QUESTIONS_FILE)

# pdata = _loadJson(POLICIES_FILE)

# q_output = {}


# def preprocessStatement(statement):
# 	prefix = "Does the privacy policy affirm"
# 	replacement = "The privacy policy affirms"
# 	processed_text = ("The privacy policy affirms" + statement[len(prefix) :])[:-1] + "."

# 	return processed_text


# for question, v in qdata.items():
# 	statment = preprocessStatement(question)
# 	for p_hash, w in v["policy_data"].items():
# 		for k, x in w.items():
# 			for y in x:
# 				print(y)
# 				if statment in q_output:
# 					q_output[statment].append(
# 						{
# 							"docId": p_hash,
# 							"start": y["substring_indices"][0],
# 							"end": y["substring_indices"][1],
# 						}
# 					)
# 				else:
# 					q_output.update(
# 						{
# 							statment: [
# 								{
# 									"docId": p_hash,
# 									"start": y["substring_indices"][0],
# 									"end": y["substring_indices"][1],
# 								}
# 							]
# 						}
# 					)

# p_output = {}
# for policy_name, v in pdata.items():
# 	for policy_hash, v in v.items():

# 		p_output[policy_hash] = v["policy_content"]

# outputfile = "./output_q.json"
# outputpolicy = "./output_p.json"


# def _saveJson(filepath, data):
# 	with open(filepath, "w", encoding="utf-8") as f:
# 		json.dump(data, f, indent=4, default=str)


# _saveJson(outputfile, q_output)
# _saveJson(outputpolicy, p_output)
# pass

{'substring': 'We implement appropriate technical and organizational security measures designed to protect personal data from loss, misuse, and unauthorized access, disclosure, alteration, or destruction.', 'substring_indices': [20045, 20234], 'method': 'FACT_EMBEDDINGS'}
{'substring': 'We implement appropriate technical and organizational security measures designed to protect personal data from loss, misuse, and unauthorized access, disclosure, alteration, or destruction.', 'substring_indices': [20045, 20234], 'method': 'FACT_EMBEDDINGS'}
{'substring': 'We implement appropriate technical and organizational security measures designed to protect personal data from loss, misuse, and unauthorized access, disclosure, alteration, or destruction.', 'substring_indices': [20045, 20234], 'method': 'FACT_EMBEDDINGS'}
{'substring': 'We implement commercially reasonable technical, administrative, and organizational measures designed to protect Personal Data from loss, misuse, and unauthorized acce